In [1]:
import pandas as pd
import os
import numpy as np
from scipy.stats import norm

In [42]:
def bs_price(S,K,sigma,r,T, year_rate = 252, option_type = "call"):
    '''
    bs 期权定价 输出为期权价格
    输入格式：
    S:标的价格
    K:行权价
    sigma:波动率
    r:无风险利率
    T:年华剩余期限
    year_rate:年化系数
    '''

    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T)) 
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == 'call':
        # return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return np.maximum(S - K, 0) if T==0 else S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    elif option_type == 'put':
        # return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        return np.maximum(K-S,0) if T==0 else K*np.exp(-r*T)*norm.cdf(-1*d2) - S*norm.cdf(-1*d1)
    else:
        print("期权类型错误")
        return -1

def bs_vega(S,K,sigma,r,T ,year_rate = 252):
    '''
    计算vega
    '''
    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(np.abs(T)) )
    return S*norm.pdf(d1)*np.sqrt(np.abs(T))
    # return S * np.sqrt(abs(T)) * np.exp(-pow(d1,2)/2) / np.sqrt(2*np.pi)

def iv_bs_bisection(S, K, r, T, price, option_type, iv_uplimit = 1.0, iv_downlimit = 0.01, precision = 3, year_rate = 365, max_iterations = 200):
    '''二分法求iv'''
    left_iv, right_iv = iv_downlimit, iv_uplimit
    # left_price = BS_price(S,K,left_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # right_price= BS_price(S,K,right_iv, r, T,  year_rate = year_rate, option_type = option_type)
    mid_iv = (left_iv + right_iv)/2
    mid_price = bs_price(S,K,mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # print(f"first, the mid price is {mid_price}, the price is {price}")
    cnt = 0
    while abs(price - mid_price) >= 0.1**precision and cnt < max_iterations:
        if mid_price < price :
            left_iv = mid_iv
        else:
            right_iv = mid_iv
        mid_iv = (left_iv + right_iv)/2
        # if mid_iv < iv_uplimit or mid_iv > iv_downlimit:
        #     break
        # print(f"cnt = {cnt}, the left is{left_iv}, the right is{right_iv}, the mid is {mid_iv}")
        cnt += 1
        mid_price = bs_price(S,K,mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    return mid_iv


def find_vol_newton( S, K, r, T, target_value, option_type, start_sigma = 0.5, precision = 3, year_rate = 252, max_iterations = 100):
    '''迭代法求iv'''
    sigma = start_sigma
    for i in range(0, max_iterations):
        bs_price_ = bs_price(S,K,sigma,r,T, year_rate = year_rate, option_type = option_type)
        vega = bs_vega(S, K, T, r, sigma, year_rate = year_rate)*100
        diff = target_value - bs_price_  # our root
        if (abs(diff) < 0.1**precision):
            return sigma
        sigma = sigma + diff/(vega) # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far


In [45]:
iv_bs_bisection(3.32, 3.4, 0.018,73, 0.33, 'call')

0.6083398558139801

In [46]:
find_vol_newton(3.32, 3.4, 0.018,73, 0.33, 'call')

0.6083396961968708

In [8]:
rate = pd.read_excel(r".\data\unrisked_rate.xlsx")
rate['date'] = pd.to_datetime(rate['date'])
rate['Value'] = rate['Value']/100 
rate.head()

,date,Value
0,2012-10-08,0.035896
1,2012-10-09,0.035942
2,2012-10-10,0.036249
3,2012-10-11,0.036252
4,2012-10-12,0.036240


In [9]:
etf50 = pd.read_excel(r'.\data\50etf_df.xlsx')
etf50 = etf50[['Date','close']]
etf50['Date'] = pd.to_datetime(etf50['Date'])
etf50.head()

,Date,close
0,2005-02-23,0.876
1,2005-02-24,0.876
2,2005-02-25,0.880
3,2005-02-28,0.872
4,2005-03-01,0.867


In [8]:
merged_option1 = pd.merge(etf50, option1, left_on = 'Date', right_on = 'date', how = 'inner')

NameError: name 'option1' is not defined

In [ ]:
merged_option1 = pd.merge(rate, option1, on = 'date', how = 'inner')

In [ ]:
merged_option1=[]

,date,Value,ptmtradeday,exe_price,open,high,low,close,call_or_put
0,2015-02-09,4.5769,28,2.2,0.1820,0.2029,0.1699,0.1826,call
1,2015-02-10,4.5696,27,2.2,0.1856,0.2144,0.1800,0.2072,call
2,2015-02-11,4.5715,26,2.2,0.2083,0.2195,0.2028,0.2107,call
3,2015-02-12,4.5540,25,2.2,0.2141,0.2143,0.1915,0.2109,call
4,2015-02-13,4.5669,24,2.2,0.2130,0.2459,0.2090,0.2090,call
5,2015-02-16,4.5716,23,2.2,0.2111,0.2219,0.2066,0.2172,call
6,2015-02-17,4.5626,22,2.2,0.2226,0.2408,0.2226,0.2296,call
7,2015-02-25,4.5924,21,2.2,0.2235,0.2250,0.1816,0.1885,call
8,2015-02-26,4.6345,20,2.2,0.1900,0.2530,0.1730,0.2530,call
9,2015-02-27,4.6482,19,2.2,0.2534,0.2739,0.2431,0.2488,call


In [27]:
csv_folder = r'.\data\50etf_option_data_csv_cleaned'
output_folder = r'.\data\unrisked_rate'
csv_list = os.listdir(csv_folder)
def change(type_name):
    if type_name == "认购":
        return "call"
    elif type_name == "认沽":
        return "put"
    else:
        return "wrong_type"
for csv_name in csv_list:
    print(csv_name)
    csv_path = csv_folder +"\\"+csv_name
    output_path = output_folder + "\\" + csv_name
    data = pd.read_csv(csv_path, encoding = 'gbk', index_col= False)
    data["date"] = pd.to_datetime(data["date"])
    data = data[['date','ptmtradeday','exe_price', 'open', 'high','low','close', 'call_or_put']]
    data['call_or_put'] = data['call_or_put'].apply(change)
    merged_option1 = pd.merge(rate, data, on = 'date', how = 'inner')
    merged_option2 = pd.merge(etf50, merged_option1, left_on = 'Date', right_on = 'date', how = 'inner')
    merged_option2 = merged_option2[['call_or_put','date','ptmtradeday','exe_price','Value', 'open', 'high','low','close_y','close_x']]
    merged_option2.columns = ['call_or_put', 'date','ptmtradeday','k', 'rate', 'open', 'high','low','close','s']
    # print(merged_option2.s)
    # merged_option2['iv'] = merged_option2.apply(lambda row : iv_bs_bisection(row['s'], row['k'], row['rate'], row['ptmtradeday'], row['close'], row['call_or_put'], iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365),  axis=1)
    merged_option2['iv'] = pd.Series
    for i in range(merged_option2.shape[1]):
        merged_option2['iv'][i] = iv_bs_bisection(np.array(merged_option2['s'][i]), np.array(merged_option2['k'][i]), np.array(merged_option2['rate'][i]), np.array(merged_option2['ptmtradeday'][i]), np.array(merged_option2['close'][i]), np.array(merged_option2['call_or_put'][i]), iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365, max_iterations= 120)
        # merged_option2['iv'][i] = find_vol_newton(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i],start_sigma = 0.1, precision = 3, year_rate = 365.0, max_iterations = 100)
        print(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i])
        print(merged_option2['iv'][i])
        # break
    merged_option2.to_csv(output_path)
    break
    # data processing
    # data.to_csv(output_path)


10000001.csv
2.331 2.2 0.045769000000000004
0.250075
2.369 2.2 0.045696
0.250075
2.38 2.2 0.045715000000000006
0.250075
2.384 2.2 0.045540000000000004
0.250075
2.39 2.2 0.045669
0.250075
2.394 2.2 0.045716
0.250075
2.411 2.2 0.045626
0.250075
2.37 2.2 0.04592399999999999
0.250075
2.45 2.2 0.046345
0.250075
2.438 2.2 0.046482
0.250075
2.441 2.2 0.046612
0.250075


C:\Users\17485\AppData\Local\Temp\ipykernel_39424\3517990051.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_option2['iv'][i] = iv_bs_bisection(np.array(merged_option2['s'][i]), np.array(merged_option2['k'][i]), np.array(merged_option2['rate'][i]), np.array(merged_option2['ptmtradeday'][i]), np.array(merged_option2['close'][i]), np.array(merged_option2['call_or_put'][i]), iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365, max_iterations= 120)


In [ ]:
merged_option2

,call_or_put,date,ptmtradeday,k,rate,open,high,low,close,s
0,call,2015-02-09,28,2.2,0.045769,0.1820,0.2029,0.1699,0.1826,2.331
1,call,2015-02-10,27,2.2,0.045696,0.1856,0.2144,0.1800,0.2072,2.369
2,call,2015-02-11,26,2.2,0.045715,0.2083,0.2195,0.2028,0.2107,2.380
3,call,2015-02-12,25,2.2,0.045540,0.2141,0.2143,0.1915,0.2109,2.384
4,call,2015-02-13,24,2.2,0.045669,0.2130,0.2459,0.2090,0.2090,2.390
5,call,2015-02-16,23,2.2,0.045716,0.2111,0.2219,0.2066,0.2172,2.394
6,call,2015-02-17,22,2.2,0.045626,0.2226,0.2408,0.2226,0.2296,2.411
7,call,2015-02-25,21,2.2,0.045924,0.2235,0.2250,0.1816,0.1885,2.370
8,call,2015-02-26,20,2.2,0.046345,0.1900,0.2530,0.1730,0.2530,2.450
9,call,2015-02-27,19,2.2,0.046482,0.2534,0.2739,0.2431,0.2488,2.438


In [ ]:
i = 0
merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i]
iv_bs_bisection(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i], iv_uplimit = 1.0, iv_downlimit = 0.01, precision = 3, year_rate = 365)

0.39671875

In [ ]:
data = pd.read_csv(r'D:\50etf期权数据与处理器\50etf_option_data_csv\10000003.csv', encoding = 'gbk',index_col=False)
# data["date"] = pd.to_datetime(data["date"])

In [ ]:

np.array(merged_option2['s'])

array([2.331, 2.369, 2.38 , 2.384, 2.39 , 2.394, 2.411, 2.37 , 2.45 ,
       2.438, 2.441, 2.364, 2.362, 2.334, 2.339, 2.397, 2.359, 2.374,
       2.448, 2.455, 2.51 , 2.543, 2.611, 2.587, 2.635, 2.676, 2.638,
       2.604])